# 2018-09-04 (Martes)

---

## Descarga librerias

```
gensim
    (Anaconda) conda install -c anaconda gensim
    (Mac o Linux) pip install --upgrade gensim

actualizar nltk
    nltk.download()
```

In [1]:
import nltk

In [ ]:
nltk.download()

## Datos no estructurados

Con datos no estructurado me refiero a cualquier colección de información que no esté en formato de base de datos y sobre la cual queramos extraer información relevante para nuestro análisis. En las siguientes clases trabajaremos extrayendo información de 3 tipos:

- HTML
- Texto
- APIs

## 1. HTML

HTML (Hypertext Markup Language) es el lenguaje estándar de páginas web. Dicho lenguaje permite estructurar y formatear documentos de texto a los cuales accedemos usualmente a través del explorador de internet (Chrome, Firefox, IExplorer, etc).

La estructura típica de un documento HTML es la siguiente:

```html
<!DOCTYPE html>
<html>
    <head>
    <title>Page Title</title>
    </head>
<body>
    <div>
        <h1>My First Heading</h1>
        <p>My first paragraph.</p>
    </div>
</body>
</html>
```


Para hacer solicitudes a páginas web, utilizaremos el paquete "requests"

Documentación: http://docs.python-requests.org/en/master/

### Ejemplo 1. Búsqueda información "I'm feeling lucky"... 

Supongamos que queremos buscar informacion sobre algo, y no sabemos nada al respecto... ¿qué hacemos hoy en día?

** lo Googleamos **

Si sabemos muy poco, en general el primer link nos servirá para informarnos... para esto Google incorporó el botón "I'm feeling lucky" 

In [18]:
import requests

terminos_a_buscar = ['carlos', 'alvarado']

search_query = '+'.join(terminos_a_buscar)

url_google = 'http://www.google.com/search?q={}&btnI'.format(search_query)

print('haciendo GET request a: ', url_google)

response = requests.get(url_google)

haciendo GET request a:  http://www.google.com/search?q=carlos+alvarado&btnI


In [19]:
#para extraer informacion de la respuesta recibida:

print('url visitada:', response.url)
print('headers:', response.headers)
print('status code:', response.status_code)
print('reason:', response.reason)

url visitada: https://es.wikipedia.org/wiki/Carlos_Alvarado_Quesada
headers: {'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Tue, 04 Sep 2018 11:54:51 GMT', 'Accept-Ranges': 'bytes', 'X-Cache': 'cp1077 hit/1, cp1087 hit/29', 'X-Analytics': 'ns=0;page_id=7580723;https=1;nocookies=1', 'Connection': 'keep-alive', 'Backend-Timing': 'D=110648 t=1535916354261781', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-Encoding': 'gzip', 'Content-Length': '23178', 'Set-Cookie': 'WMF-Last-Access=04-Sep-2018;Path=/;HttpOnly;secure;Expires=Sat, 06 Oct 2018 00:00:00 GMT, WMF-Last-Access-Global=04-Sep-2018;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sat, 06 Oct 2018 00:00:00 GMT, GeoIP=CL:RM:Santiago:-33.45:-70.67:v4; Path=/; secure; Domain=.wikipedia.org', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Content-language': 'es', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'X-Varnish': '729384415 7

In [20]:
html = response.text
print(html)

<!DOCTYPE html>
<html class="client-nojs" lang="es" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Carlos Alvarado Quesada - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Carlos_Alvarado_Quesada","wgTitle":"Carlos Alvarado Quesada","wgCurRevisionId":110373886,"wgRevisionId":110373886,"wgArticleId":7580723,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Artículos con enlaces externos rotos","Personas vivas","Wikipedia:Artículos con identificadores VIAF","Wikipedia:Artículos con identificadores ISNI","Wikipedia:Artículos con identificadores GND","Hombres","Nacidos en 1980","Presidentes de Costa Rica","Novelistas d

Existen librerías de Python que nos permiten "navegar" a través de la estructura HTML (a través de "tags") y extraer información relevante. 

La librería que usaremos aquí se llama "BeautifulSoup".

Documentación: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Tutorial webscraping con BeautifulSoup: https://www.dataquest.io/blog/web-scraping-tutorial-python/

In [15]:
#ahora automaticemos la extraccion de texto de esta lista de HTMLs

from bs4 import BeautifulSoup

textos_limpios = []

soup = BeautifulSoup(response.text, 'html.parser')

#supongamos que queremos extraer todo el texto de la seccion "body"

#la funcion "select" nos permite enfocarnos solo en esa seccion
body = soup.select('body')

#ahora "body" es una lista con todas las posibles secciones "body" (sabemos que hay solo 1, pero podrian haber mas)

print(body)


[<body class="mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject page-Computación_cuántica rootpage-Computación_cuántica skin-vector action-view"> <div class="noprint" id="mw-page-base"></div>
<div class="noprint" id="mw-head-base"></div>
<div class="mw-body" id="content" role="main">
<a id="top"></a>
<div class="mw-body-content" id="siteNotice"><!-- CentralNotice --></div><div class="mw-indicators mw-body-content">
</div>
<h1 class="firstHeading" id="firstHeading" lang="es">Computación cuántica</h1> <div class="mw-body-content" id="bodyContent">
<div class="noprint" id="siteSub">De Wikipedia, la enciclopedia libre</div> <div id="contentSub"></div>
<div id="jump-to-nav"></div> <a class="mw-jump-link" href="#mw-head">Ir a la navegación</a>
<a class="mw-jump-link" href="#p-search">Ir a la búsqueda</a>
<div class="mw-content-ltr" dir="ltr" id="mw-content-text" lang="es"><div class="mw-parser-output"><table class="noprint plainlinks ambox ambox-style" style="">
<tbody><tr>
<td cla

La función "select" nos permite una gran flexibilidad de opciones para seleccionar distintas secciones de la página. Además, existe una función similar llamada "find_all" que también nos permite seleccionar secciones de la página con distintos criterios.

Una vez que ya tengo selecionada la sección que quiero, puedo ocupar la función "get_text" para extraer el texto relevante:

In [21]:
#la funcion "get_text()" limpia todos los tags y nos deja solo el texto principal
texto = body[0].get_text()

print(texto)


 





Computación cuántica 
De Wikipedia, la enciclopedia libre 
 Ir a la navegación
Ir a la búsqueda



 
Este artículo o sección  necesita ser wikificado, por favor, edítalo para que las cumpla con las convenciones de estilo.Este aviso fue puesto el 26 de febrero de 2018.


 La esfera de Bloch es una representación de un cúbit, el bloque de construcción fundamental de los computadores cuánticos.
La computación cuántica es un paradigma de computación distinto al de la computación clásica. Se basa en el uso de cúbits en lugar de bits, y da lugar a nuevas puertas lógicas que hacen posibles nuevos algoritmos.
Una misma tarea puede tener diferente complejidad en computación clásica y en computación cuántica, lo que ha dado lugar a una gran expectación, ya que algunos problemas intratables pasan a ser tratables. Mientras que un computador clásico equivale a una máquina de Turing,[1]​ un computador cuántico equivale a una máquina de Turing cuántica.

Índice

1 Origen de la computación cuán

In [17]:

#"texto" viene con muchos caracteres "\n". Eliminemos y separemos con .
texto_limpio = texto.replace('\n', '. ')

print(texto_limpio)


 . . . . . . Computación cuántica . De Wikipedia, la enciclopedia libre .  Ir a la navegación. Ir a la búsqueda. . . .  . Este artículo o sección  necesita ser wikificado, por favor, edítalo para que las cumpla con las convenciones de estilo.Este aviso fue puesto el 26 de febrero de 2018.. . .  La esfera de Bloch es una representación de un cúbit, el bloque de construcción fundamental de los computadores cuánticos.. La computación cuántica es un paradigma de computación distinto al de la computación clásica. Se basa en el uso de cúbits en lugar de bits, y da lugar a nuevas puertas lógicas que hacen posibles nuevos algoritmos.. Una misma tarea puede tener diferente complejidad en computación clásica y en computación cuántica, lo que ha dado lugar a una gran expectación, ya que algunos problemas intratables pasan a ser tratables. Mientras que un computador clásico equivale a una máquina de Turing,[1]​ un computador cuántico equivale a una máquina de Turing cuántica.. . Índice. . 1 Origen

### Ejercicio en clase:

Escriba una funcion que tome una URL determinada y entregue el texto "limpio" de dicha URL

#### interesante, pero... ¿qué podemos hacer con esto?

1. Actualizar bases de datos con origen en información Web (por ej, tablas o datos puntuales).
1. Automatiza la extracción masiva de información (por ej, de una gran cantidad de URLs) (webscraping).
1. Automatiza la recopilación de información de internet de manera arbitraria (webcrawling).



### Ejemplo de webscraping

Primero cabe preguntarse, ¿por qué querríamos descargar mucho texto? ¿qué podemos aprender de muchas colecciones de texto?

Algunas tareas de procesamiento de texto que se han tratado de automatizar con computadores son:
- Análisis de sentimiento (marketing, política, finanzas)
- Categorización de documentos (finanzas)
- Extracción de relaciones o entidades (análisis forense)
- Sistemas de preguntas/respuestas (empresas con muchos clientes)
- Detección de plagio (academia, universidades)
- Traducción de documentos
- Resumen de contenido

En general, estos desarrollos (y otros) se enmarcan dentro de lo que se conoce como **NLP (Natural Language Processing)**, que es un área del conocimiento que intersecta la linguistica y la computación. 

Estas tareas también se han tratado con técnicas de aprendizaje estadístico (Statistical Learning), que derivan en modelos de "Inteligencia Artificial".

In [7]:
urls = [url_google,
        'https://www.cidob.org/biografias_lideres_politicos/america_central_y_caribe/costa_rica/carlos_alvarado_quesada',
        'https://es.wikipedia.org/wiki/Costa_Rica',
        'https://elpais.com/internacional/2018/05/07/america/1525706014_193325.html'
       ]

#objeto para guardar los resultados de busqueda
textos_limpios = []

#ahora iteramos para las URLs y hacemos request para cada una
for url in urls:
    texto_limpio = get_texto_from_url(url)
    
    textos_limpios.append(texto_limpio)


In [22]:
len(textos_limpios)

for t in textos_limpios:
    print(len(t))

IndexError: list index out of range

El "flujo" de trabajo **recomendado** sería guardar la información de textos (y la "metadata") en una base de datos. Por ahora omitiremos este paso y trabajaremos con los datos en memoria.

### Ejemplo de NLP

¿como podemos extraer info desde una larga colección de textos? Opcion 1. Natural Language Processing (NLP)

In [15]:
import nltk

In [16]:
textos = ' '.join(textos_limpios)

palabras_de_textos = [word for word in textos.split()]

nltk_text = nltk.Text(palabras_de_textos)

In [43]:
print(nltk_text.collocations())

fdist1 = nltk.FreqDist(nltk_text) 
fdist1.most_common(10)


Costa Rica; Costa Rica.; Artículo principal:; Archivado desde; Costa
Rica,; Palmer, 2011,; requiere |url=; |fechaacceso= requiere; Botey
Sobrado,; Costa Rica».; |url= (ayuda).; Nación. Consultado; Sobrado,
2002,; (en inglés).; «Costa Rica; América Latina; Costa Rica..; Carlos
Alvarado; Estados Unidos; así como
None


[('de', 3913),
 ('.', 2884),
 ('la', 1429),
 ('el', 1416),
 ('y', 1265),
 ('en', 1083),
 ('del', 627),
 ('a', 547),
 ('Costa', 540),
 ('que', 486)]

hmm.. Lo anterior no es muy útil porque tiene muchos "stopwords"... saquémoslos

In [44]:
from nltk.corpus import stopwords
palabras_stop = stopwords.words('spanish')

palabras_de_textos_limpios = [word for word in textos.split() if word not in palabras_stop]

nltk_text_limpio = nltk.Text(palabras_de_textos_limpios)

nltk_text_limpio.collocations()

fdist2 = nltk.FreqDist(nltk_text_limpio) 
fdist2.most_common(50)

Costa Rica; Costa Rica.; Archivado original; Artículo principal:;
Molina Palmer,; Costa Rica,; Palmer, 2011,; requiere |url=;
|fechaacceso= requiere; Botey Sobrado,; |url= (ayuda).; Costa Rica».;
Nación. Consultado; Sobrado, 2002,; (en inglés).; «Costa Rica; América
Latina; Costa Rica..; enero 2012.; Carlos Alvarado


[('.', 2884),
 ('Costa', 540),
 ('↑', 398),
 ('Rica', 296),
 ('La', 250),
 ('Consultado', 245),
 ('El', 190),
 ('país', 145),
 ('=', 132),
 ('En', 120),
 ('Rica.', 112),
 ('p.', 94),
 ('San', 81),
 ('Nacional', 77),
 ('2015.', 64),
 ('agosto', 62),
 ('}.', 62),
 ('enero', 60),
 ('Rica,', 60),
 ('{.', 60),
 ('2017.', 59),
 ('Alvarado', 58),
 ('if', 54),
 ('Carlos', 53),
 ('abril', 53),
 ('América', 53),
 ('mayo', 52),
 ('2016.', 52),
 ('Los', 50),
 ('marzo', 49),
 ('%', 49),
 ('b', 48),
 ('Archivado', 45),
 ('José', 44),
 ('Nación.', 43),
 ('original', 43),
 ('septiembre', 42),
 ('Universidad', 41),
 ('años', 41),
 ('«Costa', 40),
 ('Rica».', 40),
 ('1', 39),
 ('febrero', 39),
 ('diciembre', 39),
 ('var', 39),
 ('costarricense', 38),
 ('Artículo', 38),
 ('Instituto', 37),
 ('primera', 37),
 ('siglo', 37)]

Hm... hagamos una funcion para limpiar algunas palabras


In [19]:
def helper_clean_word(w):
        
    del_chars = ['«', ',', '↑', '-', '0', 
                 '1', '2', '3', '4', '5', 
                 '6', '7', '8', '9', '#',
                '&', '$', '(', ')', '.',
                '%', '/', ':', ';', '[',
                ']', ]
        
    for c in del_chars:
        w = w.replace(c, '')
        
    w = w.replace('á', 'a')
    w = w.replace('é', 'e')
    w = w.replace('í', 'i')
    w = w.replace('ó', 'o')
    w = w.replace('ú', 'u')
       
    return w

In [45]:
palabras_de_textos_limpios2 = [helper_clean_word(word) for word in textos.split() if word not in palabras_stop and helper_clean_word(word)]

nltk_text_limpio2 = nltk.Text(palabras_de_textos_limpios2)

nltk_text_limpio2.collocations()

fdist3 = nltk.FreqDist(nltk_text_limpio2) 
fdist3.most_common(50)

Costa Rica; Archivado original; Molina Palmer; San Jose; Articulo
principal; Estados Unidos; America Latina; requiere |url=;
|fechaacceso= requiere; Botey Sobrado; |url= ayuda; Costa Rica»;
Carlos Alvarado; Consultado agosto; Nacion Consultado; Valle Central;
Consultado enero; Luis Guillermo; Asamblea Legislativa; Consultado
febrero


[('Costa', 584),
 ('Rica', 518),
 ('La', 274),
 ('Consultado', 245),
 ('pais', 206),
 ('El', 199),
 ('=', 132),
 ('En', 121),
 ('p', 98),
 ('Nacional', 92),
 ('}', 85),
 ('San', 84),
 ('Alvarado', 78),
 ('Jose', 78),
 ('costarricense', 71),
 ('nacional', 70),
 ('Carlos', 67),
 ('{', 67),
 ('años', 66),
 ('agosto', 66),
 ('\u200b', 65),
 ('America', 65),
 ('Nacion', 62),
 ('enero', 61),
 ('abril', 58),
 ('mayo', 55),
 ('marzo', 54),
 ('if', 54),
 ('principal', 53),
 ('Los', 52),
 ('Pacifico', 51),
 ('b', 48),
 ('Universidad', 46),
 ('septiembre', 46),
 ('Central', 46),
 ('Archivado', 45),
 ('Republica', 44),
 ('diciembre', 44),
 ('original', 43),
 ('Rica»', 42),
 ('var', 41),
 ('febrero', 40),
 ('Estado', 39),
 ('primera', 39),
 ('Articulo', 39),
 ('Molina', 39),
 ('siglo', 38),
 ('Instituto', 37),
 ('poblacion', 37),
 ('Juan', 37)]

#### Ejemplo 2. Puntaje alumnos en Kattis


In [2]:
# es importante correr esta celda solo 1 vez, para evitar hacer 
# muchos GET requests a la pagina de Kattis

import requests

base_url = 'https://open.kattis.com/universities/uc.cl'

# el metodo "get" de la libreria requests nos permite hacer una solicitud "get"
# al servidor señalado. Dicha libreria nos entrega un objeto "Response",
# que incluye información sobre la respuesta que tuvo nuestra solicitud

request = requests.get(base_url)

print(request.text)

<!DOCTYPE html>

<html lang="en">
<head>
    <meta charset="UTF-8" >
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Pontifical Catholic University of Chile &ndash; Kattis, Kattis</title>

    <link href="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.10.3/css/base/minified/jquery-ui.min.css" rel="stylesheet">

    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.1/jquery.min.js"></script>
    <script src="//ajax.googleapis.com/ajax/libs/jqueryui/1.10.3/jquery-ui.min.js"></script>

    <!-- Fonts/Icons -->
    <link href="//cdnjs.cloudflare.com/ajax/libs/font-awesome/4.0.1/css/font-awesome.min.css" rel="stylesheet">

    <link href="//fonts.googleapis.com/css?family=Open+Sans:400,300,300italic,400italic,600,600italic,700,800,700italic,800italic%7CMerriweather:400,400italic,700" rel="stylesheet" type="text/css">

    <!-- Bootstrap CSS -->
    <link href="//cdnjs.cloudflare.com/ajax/

In [3]:
from bs4 import BeautifulSoup

# El objeto "BeautifulSoup", toma como parámetros el texto de la página HTML
# y el tipo de "parser" (traductor o codificador) usar

soup = BeautifulSoup(request.content, 'html.parser')

#ahora, el objeto "soup" nos permite buscar tags dentro del HTML
tags = soup.find_all('table')

#tags es una lista de objetos "Tag" (este es similar a un objeto "soup")
print(tags)

for tag in tags:
    print(type(tag))

print(type(soup))

[<table>
<tr>
<td><span>Rank</span></td>
<td><span>Score</span></td>
</tr>
<tr>
<td>161</td>
<td>129.9</td>
</tr>
</table>, <table class="table table-hover table-kattis">
<thead>
<tr>
<th class="table-min-wrap">#</th>
<th>User</th>
<th>Country</th>
<th class="table-min-wrap">Score <span class="wrapped-link"><a aria-label="Explanation of score" href="/help/ranklist#score">?</a></span></th>
</tr>
</thead>
<tbody>
<tr class="">
<td>
                                                1
                                            </td>
<td><a href="/users/martins">Martín Muñoz Cruces</a></td>
<td class="table-min-wrap">
<div class="country-flag has_flag"><a href="/countries/CHL" title="Chile"><img src="/images/countries/CHL/40px?e3bcbe8bc7c9f199da895db8e5f03f20"/></a><span class="text"><a href="/countries/CHL">Chile</a></span></div>
</td>
<td class="table-td-align-right">448.4</td>
</tr>
<tr class="">
<td>
                                                2
                                      

In [ ]:
help(BeautifulSoup)

In [ ]:
score_table = tags[1]

print(score_table)

In [ ]:
#cargo lista de usuarios a buscar
with open('../clase7_20180830/usuarios_kattis.txt', 'r', encoding='utf-8-sig') as archivo:
    alumnos = [line.strip() for line in archivo]

print('Lista de usuarios a buscar:')
print(alumnos)
print()

#selecciono las filas de la tabla
row_tags = score_table.select('tr')
print('Hay {} filas en la tabla'.format(len(row_tags)))

In [ ]:

print('Descargando datos de la tabla:...')
print()

DATA = []

#extraigo información fila por fila
for row in row_tags:
    
    #selecciono las columnas de esa fila
    col_tags = row.select('td')
    
    #extraigo el texto de cada "celda"
    aux_row = [celda.get_text().strip() for celda in col_tags]
    
    #si hay información, la guardo
    if aux_row:
        DATA.append(aux_row)

for row in DATA:
    print(row)


In [ ]:
#extraigo información relevante de los usuarios que me interesan
for num, nombre, pais, puntaje in DATA:
    if nombre in alumnos:
        print((num, nombre, pais, puntaje))